FishyScapes DATASET Computation.
This notebook computes the dataset composed of the images and the masks.

The starter images were taken from the Cityscapes dataset to which it was applied a .png image of an anomaly.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -U gdown
!gdown --folder https://drive.google.com/drive/folders/1f8Pxl8iGoRym0cagDvaweP3lbBNO2tuk?usp=sharing -O /content/cityscape --fuzzy

Retrieving folder contents
Processing file 1vKuWcc66WqiIGUPDVaha7C8w1sd13G_a gtFine.zip
Processing file 1itO7zdz4jaPL6OjuG5FvV-9tFECrdbj4 leftImg8bit.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1vKuWcc66WqiIGUPDVaha7C8w1sd13G_a
From (redirected): https://drive.google.com/uc?id=1vKuWcc66WqiIGUPDVaha7C8w1sd13G_a&confirm=t&uuid=cd6cf94c-e9a9-44c1-9ad9-8fd697bf9636
To: /content/cityscape/gtFine.zip
100% 261M/261M [00:04<00:00, 57.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1itO7zdz4jaPL6OjuG5FvV-9tFECrdbj4
From (redirected): https://drive.google.com/uc?id=1itO7zdz4jaPL6OjuG5FvV-9tFECrdbj4&confirm=t&uuid=52b501f4-d78d-44ee-a0d8-2303cf6862d1
To: /content/cityscape/leftImg8bit.zip
100% 11.6G/11.6G [02:34<00:00, 75.0MB/s]
Download completed


In [ ]:
!ls /content/cityscape

gtFine.zip  leftImg8bit.zip


In [ ]:
import zipfile

zip_gt = "/content/cityscape/gtFine.zip"
zip_img = "/content/cityscape/leftImg8bit.zip"


with zipfile.ZipFile(zip_gt, 'r') as zip_ref:
    zip_ref.extractall("/content/cityscape")

with zipfile.ZipFile(zip_img, 'r') as zip_ref:
    zip_ref.extractall("/content/cityscape")

In [ ]:
import os
import shutil
from glob import glob

# Directory of Cityscapes images and masks
img_dir = '/content/cityscape/leftImg8bit/train'
mask_dir = '/content/cityscape/gtFine/train'

# Destination Directory
save_images_dir = '/content/cityscape_sample/images'
save_masks_dir  = '/content/cityscape_sample/masks'

os.makedirs(save_images_dir, exist_ok=True)
os.makedirs(save_masks_dir, exist_ok=True)

#Find all 'train' images
image_paths = sorted(glob(os.path.join(img_dir, '*', '*_leftImg8bit.png')))

#Taking only the first 50 images
selected_images = image_paths[:50]

for i, img_path in enumerate(selected_images):

    city = os.path.basename(os.path.dirname(img_path))
    filename_base = os.path.basename(img_path).replace('_leftImg8bit.png', '')

    mask_path = os.path.join(mask_dir, city, f"{filename_base}_gtFine_labelIds.png")

    # Copying the image
    shutil.copy(img_path, os.path.join(save_images_dir, f"image_{i:03}.png"))

    # Copying the corresponding mask
    if os.path.exists(mask_path):
        shutil.copy(mask_path, os.path.join(save_masks_dir, f"mask_{i:03}.png"))
    else:
        print(f"ATTENTION: mask missing for {img_path}")

print("Operazione completata: 50 immagini e maschere salvate.")


Operazione completata: 50 immagini e maschere salvate.


In [ ]:
!pip install pillow


In [ ]:
import os
import random
from PIL import Image
import numpy as np

anomaly_dir = "/content/drive/MyDrive/gif"
output_img_dir = "/content/drive/MyDrive/fishyscapes_2/images"
output_mask_dir = "/content/drive/MyDrive/fishyscapes_2/masks"
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

img_dir_sample = "/content/cityscape_sample/images"
anomaly_paths = sorted(os.listdir(anomaly_dir))
img_paths = sorted(os.listdir(img_dir_sample))
assert len(img_paths) == 50

for i in range(50):
    img = Image.open(os.path.join(img_dir_sample, img_paths[i])).convert("RGBA")
    W, H = img.size

    mask_rgb = np.zeros((H, W, 3), dtype=np.uint8)

    anomaly = Image.open(os.path.join(anomaly_dir, random.choice(anomaly_paths))).convert("RGBA")
    w, h = anomaly.size

    scale_factor = (min(W, H) // 2) / max(w, h)
    anomaly = anomaly.resize((int(w * scale_factor), int(h * scale_factor)), Image.Resampling.LANCZOS)
    w, h = anomaly.size

    x = random.randint(0, W - w)

    # Positioning the .png in the lower half of the image with a margin of 10%
    bottom_margin = int(H * 0.10)
    y_max = H - h - bottom_margin
    y_min = max(int(H * 0.75), 0)
    if y_max < y_min:
        y_min = y_max = max(H - h - bottom_margin, 0)
    y = random.randint(y_min, y_max)

    img.paste(anomaly, (x, y), anomaly)
    alpha = np.array(anomaly.split()[-1])

    color = np.array([255, 0, 0], dtype=np.uint8)
    mask_slice = mask_rgb[y:y + h, x:x + w]
    mask_slice[alpha > 0] = color

    img.save(os.path.join(output_img_dir, f"image_{i:03}.png"))
    Image.fromarray(mask_rgb).save(os.path.join(output_mask_dir, f"mask_{i:03}.png"))
